Reference: https://medium.com/swlh/learning-to-write-language-generation-with-gpt-2-2a13fa249024#:~:text=GPT%2D2%20is%20a%20large,the%20diversity%20of%20the%20dataset.

In [1]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 11.9MB/s 
     |████████████████████████████████| 317kB 31.3MB/s 
     |████████████████████████████████| 2.9MB 57.6MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 7.4MB 46.4MB/s 
     |████████████████████████████████| 1.8MB 51.5MB/s 
     |████████████████████████████████| 1.4MB 49.8MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
     |████████████████████████████████| 112kB 59.9MB/s 
     |████████████████████████████████| 102kB 14.2MB/s 
     |████████████████████████████████| 4.5MB 45.0MB/s 
     |████████████████████████████████| 163kB 60.1MB/s 
     |████████████████████████████████| 133kB 48.4MB/s 
     |████████████████████████████████| 102kB 15.1MB/s 
     |████████████████████████████████| 890kB 48.0MB/s 
     |████████████████████████████████| 122kB 57.9MB/s 
     |████████████████████████████████| 71kB 11.9MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-

In [2]:
import pandas as pd
import numpy as np
import logging
import torch
import re
from ast import literal_eval
from simpletransformers.language_generation import LanguageGenerationModel
from simpletransformers.language_modeling import LanguageModelingModel

In [3]:
url = "https://raw.githubusercontent.com/lmu-mandy/project-rgt/bob-branch/ted_talks_en.csv"
df = pd.read_csv(url)
df = df.loc[:, ['talk_id', 'topics', 'transcript']]
df.head()

,talk_id,topics,transcript
0,1,"['alternative energy', 'cars', 'climate change...","Thank you so much, Chris. And it's truly a gre..."
1,92,"['Africa', 'Asia', 'Google', 'demo', 'economic...","About 10 years ago, I took on the task to teac..."
2,7,"['computers', 'entertainment', 'interface desi...","(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,53,"['MacArthur grant', 'activism', 'business', 'c...",If you're here today — and I'm very happy that...
4,66,"['children', 'creativity', 'culture', 'dance',...",Good morning. How are you? (Audience) Good. It...


In [4]:
sep_topics = df.topics.unique()
topics = []

for topic in sep_topics:
    for i in topic.split(","):
        topics.append(i.split("'")[1])
print(topics[0:5])

['alternative energy', 'cars', 'climate change', 'culture', 'environment']


In [5]:
unique_topics = []
# traverse for all elements 
for topic in topics: 
    # check if exists in unique_list or not 
    if topic not in unique_topics: 
            unique_topics.append(topic) 
print(unique_topics)

['alternative energy', 'cars', 'climate change', 'culture', 'environment', 'global issues', 'science', 'sustainability', 'technology', 'Africa', 'Asia', 'Google', 'demo', 'economics', 'health', 'statistics', 'global development', 'visualizations', 'math', 'computers', 'entertainment', 'interface design', 'media', 'music', 'performance', 'simplicity', 'software', 'MacArthur grant', 'activism', 'business', 'cities', 'green', 'inequality', 'politics', 'pollution', 'children', 'creativity', 'dance', 'education', 'parenting', 'teaching', 'architecture', 'collaboration', 'design', 'library', 'Christianity', 'God', 'atheism', 'comedy', 'religion', 'storytelling', 'humor', 'brain', 'cognitive science', 'consciousness', 'evolution', 'philosophy', 'happiness', 'leadership', 'motivation', 'philanthropy', 'TED Prize', 'film', 'peace', 'social change', 'art', 'movies', 'disease', 'ebola', 'disaster relief', 'invention', 'open-source', 'entrepreneur', 'piano', 'wunderkind', 'live music', 'violin', '

In [6]:
def find_topic(topic):
    """Returns a list of booleans for talks that contain a topic by index.
    
    :param topic: Topics or related topics of a talk
    """
    has_topic = []
    for t_list in df['topics']:
        if topic.lower() in literal_eval(t_list):
            has_topic.append(1)
        else:
            has_topic.append(0)
    return has_topic

In [7]:
# add columns for selected topics
df['is_science'] = find_topic('science')
df['is_technology'] = find_topic('technology')
df['is_math'] = find_topic('math')
df['is_computers'] = find_topic('computers')
df['is_engineering'] = find_topic('engineering')
df['is_ML'] = find_topic('machine learning')
df['is_software'] = find_topic('software')
df['is_statistics'] = find_topic('statistics')
df['is_cognitive_science'] = find_topic('cognitive science')
df['is_science_and_art'] = find_topic('science and art')
df['is_physics'] = find_topic('physics')
df['is_quantum_physics'] = find_topic('quantum physics')
df['is_code'] = find_topic('code')
df['is_programming'] = find_topic('programming')
df['is_chemistry'] = find_topic('chemistry')
df['is_data'] = find_topic('data')
df.head()

,talk_id,topics,transcript,is_science,is_technology,is_math,is_computers,is_engineering,is_ML,is_software,is_statistics,is_cognitive_science,is_science_and_art,is_physics,is_quantum_physics,is_code,is_programming,is_chemistry,is_data
0,1,"['alternative energy', 'cars', 'climate change...","Thank you so much, Chris. And it's truly a gre...",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,92,"['Africa', 'Asia', 'Google', 'demo', 'economic...","About 10 years ago, I took on the task to teac...",0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
2,7,"['computers', 'entertainment', 'interface desi...","(Music: ""The Sound of Silence,"" Simon & Garfun...",0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0
3,53,"['MacArthur grant', 'activism', 'business', 'c...",If you're here today — and I'm very happy that...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,66,"['children', 'creativity', 'culture', 'dance',...",Good morning. How are you? (Audience) Good. It...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# filter DataFrame to only include talks about sex, religion, and politics
df = df.loc[(df['is_science']==1) | (df['is_technology']==1) | 
            (df['is_math']==1) | (df['is_computers']==1) |
            (df['is_engineering']==1) | (df['is_ML']==1) | 
            (df['is_software'] == 1) | (df['is_statistics'] == 1) | 
            (df['is_cognitive_science'] == 1) | (df['is_science_and_art'] == 1) | 
            (df['is_physics'] == 1) | (df['is_quantum_physics'] == 1) | 
            (df['is_code'] == 1) | (df['is_programming'] == 1) | 
            (df['is_chemistry'] == 1) | df['is_data'] == 1, : ].reset_index(drop=True)

# create new DataFrames for each topic (for later use)
science_df = df.loc[(df['is_science']==1), 'talk_id':'transcript'].reset_index(drop=True)
technology_df = df.loc[(df['is_technology']==1), 'talk_id':'transcript'].reset_index(drop=True)
math_df = df.loc[(df['is_math']==1), 'talk_id':'transcript'].reset_index(drop=True)
computers_df = df.loc[(df['is_computers']==1), 'talk_id':'transcript'].reset_index(drop=True)
engineering_df = df.loc[(df['is_engineering']==1), 'talk_id':'transcript'].reset_index(drop=True)
ML_df = df.loc[(df['is_ML']==1), 'talk_id':'transcript'].reset_index(drop=True)
software_df = df.loc[(df['is_software']==1), 'talk_id':'transcript'].reset_index(drop=True)
statistics_df = df.loc[(df['is_statistics']==1), 'talk_id':'transcript'].reset_index(drop=True)
cognitive_science_df = df.loc[(df['is_cognitive_science']==1), 'talk_id':'transcript'].reset_index(drop=True)
science_and_art_df = df.loc[(df['is_science_and_art']==1), 'talk_id':'transcript'].reset_index(drop=True)
physics_df = df.loc[(df['is_physics']==1), 'talk_id':'transcript'].reset_index(drop=True)
quantum_physics_df = df.loc[(df['is_quantum_physics']==1), 'talk_id':'transcript'].reset_index(drop=True)
code_df = df.loc[(df['is_code']==1), 'talk_id':'transcript'].reset_index(drop=True)
programming_df = df.loc[(df['is_programming']==1), 'talk_id':'transcript'].reset_index(drop=True)
chemistry_df = df.loc[(df['is_chemistry']==1), 'talk_id':'transcript'].reset_index(drop=True)
data_df = df.loc[(df['is_data']==1), 'talk_id':'transcript'].reset_index(drop=True)

print('Science', science_df.shape)
print('Technology', technology_df.shape)
print('Math', math_df.shape)
print('Computers', computers_df.shape)
print('Engineering', engineering_df.shape)
print('Machine Learning', ML_df.shape)
print('Software', software_df.shape)
print('Statistics', statistics_df.shape)
print('Cognitive Science', cognitive_science_df.shape)
print('Science and Art', science_and_art_df.shape)
print('Physics', physics_df.shape)
print('Quantum Physics', quantum_physics_df.shape)
print('Code', code_df.shape)
print('Programming', programming_df.shape)
print('Chemistry', chemistry_df.shape)
print('Data', data_df.shape)

Science (993, 3)
Technology (979, 3)
Math (137, 3)
Computers (167, 3)
Engineering (156, 3)
Machine Learning (38, 3)
Software (61, 3)
Statistics (36, 3)
Cognitive Science (71, 3)
Science and Art (45, 3)
Physics (128, 3)
Quantum Physics (17, 3)
Code (37, 3)
Programming (34, 3)
Chemistry (55, 3)
Data (142, 3)


In [9]:
def combine_transcripts(transcript_list):
    """Input a list of transcripts and return them as a corpus.
    :param list_of_text: Transcript list"""
    corpus = ' '.join(transcript_list)
    return corpus

def transcripts_to_dict(df, topic_list):
    """Returns a dictionary of transcripts for each topic.
    
    :param df: DataFrame
    :param topic_list: List of topics
    """
    ted_dict = {}
    for topic in topic_list:
        # filter DataFrame to specific series and convert it to a list
        filter_string = 'is_' + str(topic)
        text_list = df.loc[(df[filter_string] == 1), 'transcript'].to_list()

        # call combine_transcripts function to return combined text
        combined_text = combine_transcripts(text_list)

        # add combined text to dict
        ted_dict[topic] = combined_text
    return ted_dict

In [10]:
# create dictionary from the DataFrame
transcript_dict = transcripts_to_dict(df, ['science', 'technology', 'math', 'computers', 'engineering', 'ML', 
                                           'software', 'statistics', 'cognitive_science', 'science_and_art', 'physics', 
                                           'quantum_physics', 'code', 'programming', 'chemistry', 'data'])

In [11]:
# construct DataFrame from dictionary
df = pd.DataFrame.from_dict(transcript_dict, orient='index')
df.rename({0: 'transcript'}, axis=1, inplace=True)
df.head()

,transcript
science,"Thank you so much, Chris. And it's truly a gre..."
technology,"Thank you so much, Chris. And it's truly a gre..."
math,"About 10 years ago, I took on the task to teac..."
computers,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
engineering,"In terms of invention, I'd like to tell you th..."


In [12]:
def clean_text(text):
    """Returns clean text.
    Removes:
        *text in square brackets & parenthesis
        *punctuation
        *words containing numbers
        *double-quotes, dashes
    """
#     text = text.lower()
    text = re.sub('[\[\(].*?[\)\]]', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[\“\–]', '', text)
    return text

In [13]:
# clean text
df['transcript'] = pd.DataFrame(df['transcript'].apply(lambda x: clean_text(x)))
science_df['transcript'] = pd.DataFrame(science_df['transcript'].apply(lambda x: clean_text(x)))
technology_df['transcript'] = pd.DataFrame(technology_df['transcript'].apply(lambda x: clean_text(x)))
math_df['transcript'] = pd.DataFrame(math_df['transcript'].apply(lambda x: clean_text(x)))
computers_df['transcript'] = pd.DataFrame(computers_df['transcript'].apply(lambda x: clean_text(x)))
engineering_df['transcript'] = pd.DataFrame(engineering_df['transcript'].apply(lambda x: clean_text(x)))
ML_df['transcript'] = pd.DataFrame(ML_df['transcript'].apply(lambda x: clean_text(x)))
software_df['transcript'] = pd.DataFrame(software_df['transcript'].apply(lambda x: clean_text(x)))
statistics_df['transcript'] = pd.DataFrame(statistics_df['transcript'].apply(lambda x: clean_text(x)))
cognitive_science_df['transcript'] = pd.DataFrame(cognitive_science_df['transcript'].apply(lambda x: clean_text(x)))
science_and_art_df['transcript'] = pd.DataFrame(science_and_art_df['transcript'].apply(lambda x: clean_text(x)))
physics_df['transcript'] = pd.DataFrame(physics_df['transcript'].apply(lambda x: clean_text(x)))
quantum_physics_df['transcript'] = pd.DataFrame(quantum_physics_df['transcript'].apply(lambda x: clean_text(x)))
code_df['transcript'] = pd.DataFrame(code_df['transcript'].apply(lambda x: clean_text(x)))
programming_df['transcript'] = pd.DataFrame(programming_df['transcript'].apply(lambda x: clean_text(x)))
chemistry_df['transcript'] = pd.DataFrame(chemistry_df['transcript'].apply(lambda x: clean_text(x)))
data_df['transcript'] = pd.DataFrame(data_df['transcript'].apply(lambda x: clean_text(x)))

In [14]:
dfs = [science_df, technology_df, math_df, computers_df, engineering_df, ML_df,
       software_df, statistics_df, cognitive_science_df, science_and_art_df, physics_df, 
       quantum_physics_df, code_df, programming_df, chemistry_df, data_df]

df = pd.concat(dfs)
df.drop_duplicates().reset_index(drop=True)
print(df.shape)
df.head()

(3096, 3)


,talk_id,topics,transcript
0,1,"['alternative energy', 'cars', 'climate change...","Thank you so much, Chris. And it's truly a gre..."
1,58,"['TED Prize', 'collaboration', 'disease', 'ebo...",I'm the luckiest guy in the world. I got to se...
2,16,"['cognitive science', 'culture', 'evolution', ...",I'd like to talk today about the two biggest s...
3,98,"['astronomy', 'biology', 'cognitive science', ...","My title: ""Queerer than we can suppose: the st..."
4,47,"['climate change', 'cosmos', 'culture', 'envir...",We've been told to go out on a limb and say so...


In [15]:
if torch.cuda.is_available():    
    device = torch.device('cuda')
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device('cpu')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [16]:
transcripts = df['transcript'].tolist()

In [17]:
index = int(len(transcripts) * 0.8)

# 80% training
with open("train.txt", "w") as f:
    for transcript in transcripts[:-index]:
        f.writelines(transcript + "\n")

# 20% testing
with open("test.txt", "w") as f:
    for transcript in transcripts[-index:]:
        f.writelines(transcript + "\n")

### GPT-2 language generation model

In [18]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = LanguageGenerationModel("gpt2", "gpt2", args={"max_length": 256})

prompts = [
    "Machine learning is"
]

for prompt in prompts:
    # Generate text using the model. Verbose set to False to prevent logging generated sequences.
    generated = model.generate(prompt, verbose=False)
    generated = '.'.join(generated[0].split('.')[:-1]) + '.'
    print('Prompt:', prompt)
    print('')
    print('Generated text:', generated)
    print('')

INFO:filelock:Lock 140582418759408 acquired on cache_dir/684fe667923972fb57f6b4dcb61a3c92763ad89882f3da5da9866baf14f2d60f.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f.lock


INFO:filelock:Lock 140582418759408 released on cache_dir/684fe667923972fb57f6b4dcb61a3c92763ad89882f3da5da9866baf14f2d60f.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f.lock
INFO:filelock:Lock 140583587367848 acquired on cache_dir/c0c761a63004025aeadd530c4c27b860ec4ecbe8a00531233de21d865a402598.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


INFO:filelock:Lock 140583587367848 released on cache_dir/c0c761a63004025aeadd530c4c27b860ec4ecbe8a00531233de21d865a402598.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


INFO:filelock:Lock 140584883549240 acquired on cache_dir/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51.lock


INFO:filelock:Lock 140584883549240 released on cache_dir/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51.lock


INFO:filelock:Lock 140582383122080 acquired on cache_dir/752929ace039baa8ef70fe21cdf9ab9445773d20e733cf693d667982e210837e.323c769945a351daa25546176f8208b3004b6f563438a7603e7932bae9025925.lock


INFO:filelock:Lock 140582383122080 released on cache_dir/752929ace039baa8ef70fe21cdf9ab9445773d20e733cf693d667982e210837e.323c769945a351daa25546176f8208b3004b6f563438a7603e7932bae9025925.lock


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Machine learning is

Generated text: Machine learning is a technique in which we can see how our learning process can change.

Learning is the process by which we are exposed to new information and experience. As we learn to move past certain challenges in the world, we find new opportunities.

We use computer science to study computer software by using a variety of means (e.g., computer simulations, reinforcement learning, and general computer science programming); the methods allow us to understand the workings of the computer while simultaneously observing and understanding the environment and the context in which information comes into existence.

In many ways, computer science is the opposite of natural science. It consists of a process of manipulating the mathematical structure and principles of mathematics. Computers have been trained, or learned, to analyze and analyze mathematical concepts with great accuracy. The system learns by studying the structure and principles 

### Fine-tuned GPT-2 model

In [19]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "train_batch_size": 20,
    "num_train_epochs": 8,
    "mlm": False,
}

model = LanguageModelingModel('gpt2', 'gpt2', args=train_args)
model.train_model("train.txt", eval_file="test.txt")
model.eval_model("test.txt")

INFO:simpletransformers.language_modeling.language_modeling_utils: Creating features from dataset file at cache_dir/


Token indices sequence length is longer than the specified maximum sequence length for this model (2608 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1834 > 1024). Running this sequence through the model will result in indexing errors


INFO:simpletransformers.language_modeling.language_modeling_utils: Saving features into cached file cache_dir/gpt2_cached_lm_126_train.txt
INFO:simpletransformers.language_modeling.language_modeling_model: Training started


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.language_modeling.language_modeling_model: Training of gpt2 model complete. Saved to outputs/.
INFO:simpletransformers.language_modeling.language_modeling_utils: Creating features from dataset file at cache_dir/


Token indices sequence length is longer than the specified maximum sequence length for this model (1747 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4054 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2697 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3534 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3320 > 1024). Running this sequence through the model will result in indexing errors


INFO:simpletransformers.language_modeling.language_modeling_utils: Saving features into cached file cache_dir/gpt2_cached_lm_126_test.txt


INFO:simpletransformers.language_modeling.language_modeling_model:{'eval_loss': 3.238466412868221, 'perplexity': tensor(25.4946)}


{'eval_loss': 3.238466412868221, 'perplexity': tensor(25.4946)}

In [21]:
# generate text using fine-tuned GPT-2 model
text_generator = LanguageGenerationModel("gpt2", "gpt2", args={"max_length": 256})

prompts = [
    "Machine learning is"
]

for prompt in prompts:
    generated = text_generator.generate(prompt, verbose=False)
    generated = '.'.join(generated[0].split('.')[:-1]) + '.'
    print('Prompt:', prompt)
    print('')
    print('Generated text:', generated)
    print('')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Machine learning is

Generated text: Machine learning is a complex and complex, multi-faceted endeavor, so our expertise, and the ability to implement many different kinds of technology for this task, has been extremely fruitful.

As you know, when I was young in the 1980s, I had no interest in programming. I thought computers were boring, and I was so excited that I made this video (it's one of many that I have worked on together over the years) of an IBM engineer who made a pretty cool computer called CursorControl, a machine that has some of the following characteristics:

A very basic set of CursorControl commands that you can quickly understand:

CursorControl is really simple and intuitive, but it also allows you to take shortcuts, check the current time or position, add to a file, or change the cursor position with the click of a key.

It's also the most popular CursorControl in the world.

It's also easy to learn, and the code is really easy to understand! And unlike so